In [ ]:
# (langchain 사용)
# 1. 문서 내용 읽기
# 2. 문서 쪼개기
#     - 왜? 토큰수 초과로 답변을 생성하지 못할 수 있다.
#     - input이 길면 답변 생성이 오래걸린다.
# 3. 임베딩 => 벡터 데이터베이스에 저장
# 4. 질문이 있을 때 벡터 데이터베이스에서 유사도 높은 데이터 찾기
# 5. 유사도 검색으로 가져온 문서를 LLM에 전달하여 답변 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 청크 하나가 가질 수 있는 토큰 수
    chunk_overlap=200, # 청크 사이즈의 20% 만큼 중복. 왜 중복필요? => 유사도 검색 시 원하는 답변이 나올 수 있는 확률을 높이기 위함 (문맥 유지)
)

loader = Docx2txtLoader("tax.docx") # 문서 읽기
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서 쪼개기

In [11]:
# 임베딩
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [12]:
# 벡터 데이터베이스에 저장
from langchain_chroma import Chroma

# 크로마는 기본적으로 in-memory database를 사용하기 때문에 질문, 답변 데이터가 손실될 수 있음. 때문에 창을 닫으면 데이터가 사라진다.
# 이를 방지하기 위해 persist_directory를 추가해 임베딩 결과를 저장한다.
# collection_name은 컬럼 이름이다.
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma_tax",persist_directory="./chroma")

database = Chroma(embedding_function=embedding, collection_name="chroma-tax", persist_directory="./chroma")

In [14]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3) # 유사도 검색

In [15]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [10]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: c:\Users\hanjo\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [16]:
from langchain_classic import hub # langchain은 deprecated 됨

prompt = hub.pull('rlm/rag-prompt')


In [ ]:
from langchain_classic.chains import RetrievalQA

# LangChain 형태로 질문 답변 체인 생성
qa_chain = RetrievalQA.from_chain_type(llm, retriever=database.as_retriever(), chain_type_kwargs={"prompt": prompt})

In [18]:
ai_message = qa_chain({"query": query})

C:\Users\hanjo\AppData\Local\Temp\ipykernel_49076\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  ai_message = qa_chain({"query": query})


In [19]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만 주어진 문맥만으로는 연봉 5천만 원인 직장인의 소득세를 정확하게 계산할 수 없습니다. 소득세는 과세 기준, 공제 항목 및 개인의 세율 등에 따라 다르기 때문에 추가 정보가 필요합니다.'}